## Kbeta

In [ ]:
# 导入相关的包
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import spectrochempy as scp
from matplotlib import gridspec, ticker
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from spectrochempy import Coord, NDDataset, ur

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#0077BB", "#33BBEE", "#009988", "#EE7733", "#CC3311", "#EE3377", "#BBBBBB"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

### Version 6

#### 读取所有数据，并 denoise

In [ ]:
# 读取数据文件夹
path_data_folder = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\ExSitu\αMnO2\Kbeta\Data\2023-CLAESS"  # noqa: RUF001
)
path_filelist = []
for item in path_data_folder.iterdir():
    if item.is_dir():
        file_dir = Path.joinpath(item, r"Mn")
        path_filelist.append(file_dir)
display(path_filelist)

In [ ]:
# 基线校准
blc = scp.Baseline(
    log_level="INFO",
    model="polynomial",  # use a polynomial model
    order="linear",  # with linear method
    ranges=([6462.0, 6463.0], [6510.0, 6511.0]),
)

# 平滑
filter = scp.Filter(method="avg", size=3)

In [ ]:
%matplotlib inline
from itertools import chain

results, nomral_spe, nomral_spe_smooth, headers = [], [], [], []
for path_file in path_filelist:
    path_out_A = Path.joinpath(path_out, f"{path_file.parts[-2]}")  # noqa: N816
    path_out_A.mkdir(parents=True, exist_ok=True)

    txt_data = []
    filelist = list(path_file.glob(r"*.txt"))
    # display(filelist)  # noqa: ERA001
    for filetxt in filelist:
        data = pd.read_csv(filetxt, comment="#", sep=r"\s+", header=None)
        txt_data.append(data)
    txt_data = pd.concat(
        txt_data,
        axis=1,
        ignore_index=True,
    )
    txt_data.to_csv(Path.joinpath(path_out_A, f"{path_file.parts[-2]}_raw_all.csv"), index=False, header=True)

    data_scp = NDDataset(
        data=txt_data.iloc[:, 1::2].T.values,
        author="Cheng Liu",
        description="Kbeta of Mn, ALBA",
        history="creation",
    )
    data_scp.x = Coord(
        txt_data.iloc[:, 0].values,
        name="Energy",
        units=ur.eV,
    )
    data_scp.y = Coord(
        np.arange((txt_data.shape[1] // 2)),
        name="numbers",
    )

    # PCA 重构数据
    recon_scp_data = scp.denoise(data_scp, ratio=99.5, log_level="ERROR")
    # recon_scp_data.plot(clear=True)  # noqa: ERA001

    # 原始数据，平均谱线，#1
    # 平均化
    recon_data_mean = recon_scp_data.mean(dim="y", keepdim=True)
    # recon_data_mean.plot(clear=True)  # noqa: ERA001
    # 基线校准
    _ = blc.fit(recon_data_mean)  # fit the baseline
    recon_data_mean = blc.corrected  # get the corrected dataset
    # 归一化
    inttrapz_area = scipy.integrate.trapezoid(y=recon_data_mean.data, x=recon_data_mean.x.data)
    recon_data_mean_normal = np.divide(recon_data_mean, inttrapz_area)
    # recon_data_mean_normal.plot(clear=True)  # noqa: ERA001

    # 基线校准
    _ = blc.fit(recon_scp_data)  # fit the baseline
    scp_baseline = blc.baseline
    scp_corrected = blc.corrected  # get the corrected dataset
    # _ = scp_corrected.plot(clear=True)  # noqa: ERA001

    # 基线校准后，平均谱线，#2
    # 平均化
    scp_corrected_mean = scp_corrected.mean(dim="y", keepdim=True)
    # 基线校准
    _ = blc.fit(scp_corrected_mean)  # fit the baseline
    scp_corrected_mean = blc.corrected  # get the corrected dataset
    # 归一化
    inttrapz_area = scipy.integrate.trapezoid(y=scp_corrected_mean.data, x=scp_corrected_mean.x.data)
    scp_corrected_mean_normal = np.divide(scp_corrected_mean, inttrapz_area)
    # scp_corrected_mean_normal.plot(clear=True)  # noqa: ERA001

    # AA 归一化
    inttrapz_area = scipy.integrate.trapezoid(y=scp_corrected.data, x=scp_corrected.x.data)
    for i in range(scp_corrected.shape[0]):
        scp_corrected[i, :] = np.divide(scp_corrected[i, :], inttrapz_area[i])

    # AA 保存数据
    (
        pd.concat(
            [pd.Series(scp_corrected.x.data), pd.DataFrame(scp_corrected.data).T],
            axis=1,
            ignore_index=True,
        ).to_csv(
            Path.joinpath(path_out_A, f"{path_file.parts[-2]}_normal_all.csv"),
            index=False,
            header=True,
        )
    )

    # # # 寻峰，以及 std 分布
    # # 比较几个操作后 前后的变化
    # def AAplot(X1, X2, X3, label=None, xlim=None):
    #     X1.plot(color="b", label="original", clear=True)
    #     X2.plot(color="g", label="corrected", clear=False)
    #     X3.plot(clear=False, color="r", ls="-", lw=1.5, label=label)

    #     diff = X1 - X3
    #     # s = round(diff.std(dim=-1, ddof=1).values, 2)
    #     s = diff.std(dim=-1, ddof=1).values
    #     ax = diff.plot(clear=False, ls="-", lw=1, label=f"difference (std={s})")
    #     ax.legend(loc="best", fontsize=10)
    #     if xlim is not None:
    #         ax.set_xlim(xlim)
    # # scp.show()

    # 平滑曲线，#3
    recon_data_mean_smooth = filter.transform(recon_data_mean)
    scp_corrected_mean_smooth = filter.transform(scp_corrected_mean)
    # 基线校准
    _ = blc.fit(recon_data_mean_smooth)  # fit the baseline
    recon_data_mean_smooth = blc.corrected  # get the corrected dataset
    _ = blc.fit(scp_corrected_mean_smooth)  # fit the baseline
    scp_corrected_mean_smooth = blc.corrected  # get the corrected dataset
    # 归一化
    inttrapz_area = scipy.integrate.trapezoid(y=recon_data_mean_smooth.data, x=recon_data_mean_smooth.x.data)
    recon_data_mean_smooth_normal = np.divide(recon_data_mean_smooth, inttrapz_area)
    # recon_data_mean_smooth_normal.plot(clear=True)  # noqa: ERA001
    inttrapz_area = scipy.integrate.trapezoid(y=scp_corrected_mean_smooth.data, x=scp_corrected_mean_smooth.x.data)
    scp_corrected_mean_smooth_normal = np.divide(scp_corrected_mean_smooth, inttrapz_area)
    # scp_corrected_mean_smooth_normal.plot(clear=True)  # noqa: ERA001

    # 确认了平滑的结果比较好；而且上述的前后平均谱线，基线校准后，归一化后，结果是一样的
    # AAplot(recon_data_mean_normal, scp_corrected_mean_normal, recon_data_mean_smooth_normal, label="Moving average (5 points)", xlim=(6462, 6510))
    # AAplot(recon_data_mean_normal, scp_corrected_mean_normal, scp_corrected_mean_smooth_normal, label="Moving average (5 points)", xlim=(6462, 6510))

    # 保存数据
    headers.append([r"Energy", f"{path_file.parts[-2]}"])
    # 非平滑的曲线
    data_normal = pd.DataFrame([scp_corrected_mean_normal.x.data, scp_corrected_mean_normal.squeeze().data]).T
    nomral_spe.append(data_normal)

    # 平滑的曲线
    data_smooth = pd.DataFrame([
        scp_corrected_mean_smooth_normal.x.data,
        scp_corrected_mean_smooth_normal.squeeze().data,
    ]).T
    nomral_spe_smooth.append(data_smooth)

    # 寻峰
    # 非平滑的结果
    peakslist = [s.find_peaks(distance=10)[0].x.data for s in scp_corrected[:, 6490.0:6500.0]]
    peakslist = pd.DataFrame(peakslist)
    peakstd0 = peakslist.std(ddof=0)
    peakmean0 = peakslist.mean()

    # 平滑后的结果
    scp_corrected_smooth = filter.transform(scp_corrected)
    peakslist = [s.find_peaks(distance=10)[0].x.data for s in scp_corrected_smooth[:, 6490.0:6500.0]]
    peakslist = pd.DataFrame(peakslist)
    peakstd = peakslist.std(ddof=0)
    peakmean = peakslist.mean()

    results.append([f"{path_file.parts[-2]}", peakmean[0], peakstd[0], peakmean0[0], peakstd0[0]])

headers = list(chain.from_iterable(headers))
pd.DataFrame(results).to_csv(
    Path.joinpath(path_out, r"peak_list.csv"),
    index=False,
    header=[r"samples", r"peak", r"peak_std", r"peak_smooth", r"peak_std_smooth"],
)
pd.concat(nomral_spe, ignore_index=True, axis=1).to_csv(
    Path.joinpath(path_out, r"spectrum_normal_all.csv"), header=headers, index=False
)
pd.concat(nomral_spe_smooth, ignore_index=True, axis=1).to_csv(
    Path.joinpath(path_out, r"spectrum_normal_smooth_all.csv"), header=headers, index=False
)

In [ ]:
# 读取标样 MnO2 的数据
for item in path_out.iterdir():
    if (item.is_dir()) and (
        item.parts[-1]
        in [
            "R5_MnO2",
        ]
    ):
        ref = pd.read_csv(
            Path.joinpath(item, f"{item.parts[-1]}_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
        )
        ref_scp = NDDataset(
            data=ref.iloc[:, 1:].T.values,
            title=r"Absorption",
            name=f"{item.parts[-1]}",
        )
        ref_scp.x = Coord(
            ref.iloc[:, 0].values,
            name="Energy",
            units=ur.eV,
        )
        ref_scp.y = Coord(
            np.arange((ref.shape[1] - 1)),
            name="numbers",
        )
        # 平滑
        ref_scp_smooth = filter.transform(ref_scp)

# 读取其他的所有数据
results = []
for item in path_out.iterdir():
    if item.is_dir():
        data = pd.read_csv(
            Path.joinpath(item, f"{item.parts[-1]}_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
        )
        scp_data = NDDataset(
            data=data.iloc[:, 1:].T.values,
            author="Cheng Liu",
            description="Kbeta of Mn, ALBA",
            history="creation",
        )
        scp_data.x = Coord(
            data.iloc[:, 0].values,
            name="Energy",
            units=ur.eV,
        )
        scp_data.y = Coord(
            np.arange((data.shape[1] - 1)),
            name="numbers",
        )
        # 平滑
        scp_data_smooth = filter.transform(scp_data)

        # IDA 面积
        integral_results = []
        for ref_spectrum in ref_scp:
            for scp_spectrum in scp_data:
                # 计算绝对差值
                diff = np.abs(scp_spectrum - ref_spectrum)
                # 使用梯形法则进行积分
                integral = np.trapz(diff.data, diff.x.data)
                integral_results.append(integral)
        integral_results = pd.DataFrame(integral_results)
        area_mean = integral_results.mean()
        area_std = integral_results.std(ddof=0)

        # IDA 面积
        integral_results = []
        for ref_spectrum in ref_scp_smooth:
            for scp_spectrum in scp_data_smooth:
                # 计算绝对差值
                diff = np.abs(scp_spectrum - ref_spectrum)
                # 使用梯形法则进行积分
                integral = np.trapz(diff.data, diff.x.data)
                integral_results.append(integral)

        integral_results = pd.DataFrame(integral_results)
        area_mean0 = integral_results.mean()
        area_std0 = integral_results.std(ddof=0)
        results.append([f"{item.parts[-1]}", area_mean[0], area_std[0], area_mean0[0], area_std0[0]])
pd.DataFrame(results).to_csv(
    Path.joinpath(path_out, r"area_list.csv"),
    index=False,
    header=[r"samples", r"area", r"area_std", r"area_smooth", r"area_std_smooth"],
)

In [ ]:
# 磁矩转换
from uncertainties import ufloat

# Load CSV
df = pd.read_csv(Path.joinpath(path_out, r"area_list.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(
    how="all", axis=1
)


# Define the function, but only take one argument: the row
def calculate_ratios(row, data):
    Mn2 = ufloat(data.iloc[2, 1], data.iloc[2, 2])
    Mn20 = ufloat(data.iloc[2, 3], data.iloc[2, 4])
    Mn4 = ufloat(data.iloc[4, 1], data.iloc[4, 2])
    Mn40 = ufloat(data.iloc[4, 3], data.iloc[4, 4])

    A = ufloat(row["area"], row["area_std"])
    A0 = ufloat(row["area_smooth"], row["area_std_smooth"])
    X = 3 + (A - Mn4) * 2 / (Mn2 - Mn4)
    X0 = 3 + (A0 - Mn40) * 2 / (Mn20 - Mn40)

    return pd.Series({
        "X": X.nominal_value,
        "X_err": X.std_dev,
        "X0": X0.nominal_value,
        "X0_err": X0.std_dev,
    })


# Apply with an extra argument using a lambda
results = df.apply(lambda row: calculate_ratios(row, df), axis=1)

# Combine and save
final_df = pd.concat([df, results], axis=1)
final_df.to_csv(Path.joinpath(path_out, r"area_list.csv"), sep=",", header=True, index=False)

In [ ]:
# 读取标样 MnO2 的数据，只用平均的 MnO2 数据
ref = pd.read_csv(
    Path.joinpath(path_out, r"spectrum_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
).iloc[:, 8:10]
ref_scp = NDDataset(
    data=ref.iloc[:, 1].T.values,
    title=r"Absorption",
    name=f"{ref.columns[1]}",
)
ref_scp.x = Coord(
    ref.iloc[:, 0].values,
    name="Energy",
    units=ur.eV,
)
# 平滑
ref_scp_smooth = filter.transform(ref_scp)

# 读取其他的所有数据
results = []
for item in path_out.iterdir():
    if item.is_dir():
        data = pd.read_csv(
            Path.joinpath(item, f"{item.parts[-1]}_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
        )
        scp_data = NDDataset(
            data=data.iloc[:, 1:].T.values,
            author="Cheng Liu",
            description="Kbeta of Mn, ALBA",
            history="creation",
        )
        scp_data.x = Coord(
            data.iloc[:, 0].values,
            name="Energy",
            units=ur.eV,
        )
        scp_data.y = Coord(
            np.arange((data.shape[1] - 1)),
            name="numbers",
        )
        # 平滑
        scp_data_smooth = filter.transform(scp_data)

        # IDA 面积
        integral_results = []
        for scp_spectrum in scp_data:
            # 计算绝对差值
            diff = np.abs(scp_spectrum - ref_scp)
            # 使用梯形法则进行积分
            integral = np.trapz(diff.data, diff.x.data)
            integral_results.append(integral)

        integral_results = pd.DataFrame(integral_results)
        area_mean = integral_results.mean()
        area_std = integral_results.std(ddof=0)

        # IDA 面积
        integral_results = []
        for scp_spectrum in scp_data_smooth:
            # 计算绝对差值
            diff = np.abs(scp_spectrum - ref_scp_smooth)
            # 使用梯形法则进行积分
            integral = np.trapz(diff.data, diff.x.data)
            integral_results.append(integral)

        integral_results = pd.DataFrame(integral_results)
        area_mean0 = integral_results.mean()
        area_std0 = integral_results.std(ddof=0)
        results.append([f"{item.parts[-1]}", area_mean[0], area_std[0], area_mean0[0], area_std0[0]])
pd.DataFrame(results).to_csv(
    Path.joinpath(path_out, r"area_list_2.csv"),
    index=False,
    header=[r"samples", r"area", r"area_std", r"area_smooth", r"area_std_smooth"],
)

In [ ]:
# 磁矩转换
from uncertainties import ufloat

# Load CSV
df = pd.read_csv(Path.joinpath(path_out, r"area_list_2.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(
    how="all", axis=1
)


# Define the function, but only take one argument: the row
def calculate_ratios(row, data):
    Mn2 = ufloat(data.iloc[2, 1], data.iloc[2, 2])
    Mn20 = ufloat(data.iloc[2, 3], data.iloc[2, 4])
    Mn4 = ufloat(data.iloc[4, 1], data.iloc[4, 2])
    Mn40 = ufloat(data.iloc[4, 3], data.iloc[4, 4])

    A = ufloat(row["area"], row["area_std"])
    A0 = ufloat(row["area_smooth"], row["area_std_smooth"])
    X = 3 + (A - Mn4) * 2 / (Mn2 - Mn4)
    X0 = 3 + (A0 - Mn40) * 2 / (Mn20 - Mn40)

    return pd.Series({
        "X": X.nominal_value,
        "X_err": X.std_dev,
        "X0": X0.nominal_value,
        "X0_err": X0.std_dev,
    })


# Apply with an extra argument using a lambda
results = df.apply(lambda row: calculate_ratios(row, df), axis=1)

# Combine and save
final_df = pd.concat([df, results], axis=1)
final_df.to_csv(Path.joinpath(path_out, r"area_list_2.csv"), sep=",", header=True, index=False)


##### 画图，非光滑的结果

In [ ]:
# 读取数据文件夹以及文件
xes_peak = pd.read_csv(Path.joinpath(path_out, r"peak_list.csv"), comment="#", sep=r",", header=0, index_col=None)
xes_area = pd.read_csv(Path.joinpath(path_out, r"area_list_2.csv"), comment="#", sep=r",", header=0, index_col=None)
spectrum = pd.read_csv(
    Path.joinpath(path_out, r"spectrum_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
)

diff_spectrum = spectrum.copy(deep=True)
odd_columns = diff_spectrum.columns[1::2]
diff_spectrum[odd_columns] = diff_spectrum[odd_columns].sub(diff_spectrum.iloc[:, 9], axis=0)

In [ ]:
# 画图
%matplotlib inline

fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

labels = [
    r"R1_MnOOH",
    r"R2_ZnMn2O4",
    r"R3_MnO",
    r"R4_Mn2O3",
    r"R5_MnO2",
    r"S1_pristine",
    r"S2_1stDisch",
    r"S3_1stHCh_1p53V",
    r"S4_1stHCh_1p63V",
    r"S5_1stCh",
    r"S6_2ndDisch_1p3V",
    r"S7_2ndDisch",
]
colormap = ListedColormap(mpl.colormaps["sunset"](np.linspace(0, 1, spectrum.shape[1] // 2 - 1)), name=r"colormap")

# 图 A: Spectrum
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

# 多线叠加
for i in range(spectrum.shape[1] // 2 - 1):
    ax.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=2,
        alpha=1 - 0.01 * i,
    )
    ax.plot(
        diff_spectrum.iloc[:, 0 + 2 * i],
        diff_spectrum.iloc[:, 1 + 2 * i] - 0.01,
        lw=1,
        label=None,
        color=colormap.colors[i],  # type: ignore
        zorder=2,
        alpha=1 - 0.01 * i,
    )

ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.set_xlim(6465, 6510)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5, offset=0))
ax.set_ylabel(ylabel=r"Relative Intensity (arb.u.)", fontsize=11, labelpad=0.0)
ax.set_ylim(-0.04, 0.16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.01, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.5,
)

axins = ax.inset_axes((0.78, 0.32, 0.2, 0.65))
for i in range(spectrum.shape[1] // 2 - 1):
    axins.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=0,
        alpha=1 - 0.01 * i,
    )
axins.set_xlim(6491, 6495)
axins.set_axis_off()
ax.text(
    -0.22,
    1.0,
    r"a",
    weight="bold",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=13,
)

# 图 B，Peak, area + std
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.2, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.errorbar(
    x=np.arange(len(labels)),
    y=xes_peak.iloc[:, 1],
    yerr=xes_peak.iloc[:, 2],
    c=colors[0],
    fmt="o-",
    linewidth=1,
    capsize=4,
)

ax.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax.set_ylabel(r"Peak Positions (eV)", fontsize=11, labelpad=10.0)
ax.set_ylim(6492.0, 6493.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.1))
formatter = ticker.ScalarFormatter(useOffset=6490)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)

arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}
ax.annotate(
    r" ", xy=(0.2, 0.5), xycoords="axes fraction", xytext=(0, 0.4), textcoords="axes fraction", arrowprops=arrowprops
)

ax2 = ax.twinx()
ax2.set_position((0.2, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.errorbar(
    x=np.arange(len(labels)),
    y=xes_area.iloc[:, 5],
    yerr=xes_area.iloc[:, 6],
    c=colors[2],
    fmt="s-",
    linewidth=1,
    capsize=4,
    alpha=0.5,
)
ax2.set_xlim(-0.5, 11.5)

ax2.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax2.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax2.set_ylabel(r"Local Magnetic Moment ($\mathrm{\mu _B}$)", fontsize=11)  # Total Magnetization
ax2.set_ylim(2.5, 5.5)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0.1))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, left=False, right=True)

arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.5,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
}

ax2.annotate(
    r" ",
    xy=(0.8, 0.28),
    xycoords="axes fraction",
    xytext=(1.0, 0.35),
    textcoords="axes fraction",
    arrowprops=arrowprops,
)
ax.text(
    -0.18,
    1.0,
    r"b",
    transform=ax.transAxes,
    fontsize=14,
    va="center",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
)

plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_300_V6_1.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_1.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_1.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.gcf().set_facecolor("white")
plt.show()

##### 画图，光滑的结果

In [ ]:
# 读取数据文件夹以及文件
xes_peak = pd.read_csv(Path.joinpath(path_out, r"peak_list.csv"), comment="#", sep=r",", header=0, index_col=None)
xes_area = pd.read_csv(Path.joinpath(path_out, r"area_list_2.csv"), comment="#", sep=r",", header=0, index_col=None)
spectrum = pd.read_csv(
    Path.joinpath(path_out, r"spectrum_normal_smooth_all.csv"), comment="#", sep=r",", header=0, index_col=None
)

diff_spectrum = spectrum.copy(deep=True)
odd_columns = diff_spectrum.columns[1::2]
diff_spectrum[odd_columns] = diff_spectrum[odd_columns].sub(diff_spectrum.iloc[:, 9], axis=0)

In [ ]:
# 画图
%matplotlib inline

fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

labels = [
    r"R1_MnOOH",
    r"R2_ZnMn2O4",
    r"R3_MnO",
    r"R4_Mn2O3",
    r"R5_MnO2",
    r"S1_pristine",
    r"S2_1stDisch",
    r"S3_1stHCh_1p53V",
    r"S4_1stHCh_1p63V",
    r"S5_1stCh",
    r"S6_2ndDisch_1p3V",
    r"S7_2ndDisch",
]
colormap = ListedColormap(mpl.colormaps["sunset"](np.linspace(0, 1, spectrum.shape[1] // 2 - 1)), name=r"colormap")

# 图 A: Spectrum
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

# 多线叠加
for i in range(spectrum.shape[1] // 2 - 1):
    ax.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=2,
        alpha=1 - 0.01 * i,
    )
    ax.plot(
        diff_spectrum.iloc[:, 0 + 2 * i],
        diff_spectrum.iloc[:, 1 + 2 * i] - 0.01,
        lw=1,
        label=None,
        color=colormap.colors[i],  # type: ignore
        zorder=2,
        alpha=1 - 0.01 * i,
    )

ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.set_xlim(6465, 6510)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5, offset=0))
ax.set_ylabel(ylabel=r"Relative Intensity (arb.u.)", fontsize=11, labelpad=0.0)
ax.set_ylim(-0.04, 0.16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.01, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.5,
)

axins = ax.inset_axes((0.78, 0.32, 0.2, 0.65))
for i in range(spectrum.shape[1] // 2 - 1):
    axins.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=0,
        alpha=1 - 0.01 * i,
    )
axins.set_xlim(6491, 6495)
axins.set_axis_off()
ax.text(
    -0.22,
    1.0,
    r"a",
    weight="bold",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=13,
)

# 图 B，Peak, area + std
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.2, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.errorbar(
    x=np.arange(len(labels)),
    y=xes_peak.iloc[:, 3],
    yerr=xes_peak.iloc[:, 4],
    c=colors[1],
    fmt="o-",
    linewidth=1,
    capsize=4,
)

ax.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax.set_ylabel(r"Peak Positions (eV)", fontsize=11, labelpad=10.0)
ax.set_ylim(6492.0, 6493.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.1))
formatter = ticker.ScalarFormatter(useOffset=6490)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)

arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}
ax.annotate(
    r" ", xy=(0.2, 0.5), xycoords="axes fraction", xytext=(0, 0.4), textcoords="axes fraction", arrowprops=arrowprops
)

ax2 = ax.twinx()
ax2.set_position((0.2, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.errorbar(
    x=np.arange(len(labels)),
    y=xes_area.iloc[:, 7],
    yerr=xes_area.iloc[:, 8],
    c=colors[3],
    fmt="s-",
    linewidth=1,
    capsize=4,
    alpha=0.5,
)
ax2.set_xlim(-0.5, 11.5)

ax2.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax2.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax2.set_ylabel(r"Local Magnetic Moment ($\mathrm{\mu _B}$)", fontsize=11)  # Total Magnetization
ax2.set_ylim(2.5, 5.5)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0.1))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, left=False, right=True)

arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.5,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
}
ax2.annotate(
    r" ",
    xy=(0.8, 0.28),
    xycoords="axes fraction",
    xytext=(1.0, 0.35),
    textcoords="axes fraction",
    arrowprops=arrowprops,
)
ax.text(
    -0.18,
    1.0,
    r"b",
    transform=ax.transAxes,
    fontsize=14,
    va="center",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
)

plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_300_V6_2.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_2.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_2.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.gcf().set_facecolor("white")
plt.show()

##### 最后的结果，画图，非光滑的 Energy peak + 光滑的 std 结果

In [ ]:
# 读取数据文件夹以及文件
xes_peak = pd.read_csv(Path.joinpath(path_out, r"peak_list.csv"), comment="#", sep=r",", header=0, index_col=None)
xes_area = pd.read_csv(Path.joinpath(path_out, r"area_list_2.csv"), comment="#", sep=r",", header=0, index_col=None)
spectrum = pd.read_csv(
    Path.joinpath(path_out, r"spectrum_normal_all.csv"), comment="#", sep=r",", header=0, index_col=None
)

diff_spectrum = spectrum.copy(deep=True)
odd_columns = diff_spectrum.columns[1::2]
diff_spectrum[odd_columns] = diff_spectrum[odd_columns].sub(diff_spectrum.iloc[:, 9], axis=0)

In [ ]:
# 画图
%matplotlib inline

fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

labels = [
    r"R1_MnOOH",
    r"R2_ZnMn2O4",
    r"R3_MnO",
    r"R4_Mn2O3",
    r"R5_MnO2",
    r"S1_pristine",
    r"S2_1stDisch",
    r"S3_1stHCh_1p53V",
    r"S4_1stHCh_1p63V",
    r"S5_1stCh",
    r"S6_2ndDisch_1p3V",
    r"S7_2ndDisch",
]
colormap = ListedColormap(mpl.colormaps["sunset"](np.linspace(0, 1, spectrum.shape[1] // 2 - 1)), name=r"colormap")

# 图 A: Spectrum
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

# 多线叠加
for i in range(spectrum.shape[1] // 2 - 1):
    ax.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],
        zorder=2,
        alpha=1 - 0.01 * i,
    )
    ax.plot(
        diff_spectrum.iloc[:, 0 + 2 * i],
        diff_spectrum.iloc[:, 1 + 2 * i] - 0.01,
        lw=1,
        label=None,
        color=colormap.colors[i],
        zorder=2,
        alpha=1 - 0.01 * i,
    )

ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.set_xlim(6465, 6510)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5, offset=0))
ax.set_ylabel(ylabel=r"Relative Intensity (arb.u.)", fontsize=11, labelpad=0.0)
ax.set_ylim(-0.04, 0.16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.01, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.5,
)

axins = ax.inset_axes((0.78, 0.32, 0.2, 0.65))
for i in range(spectrum.shape[1] // 2 - 1):
    axins.plot(
        spectrum.iloc[:, 0 + 2 * i],
        spectrum.iloc[:, 1 + 2 * i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=0,
        alpha=1 - 0.01 * i,
    )
axins.set_xlim(6491, 6495)
axins.set_axis_off()
ax.text(
    -0.22,
    1.0,
    r"a",
    weight="bold",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=13,
)

# 图 B，Peak, area + std
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.2, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.errorbar(
    x=np.arange(len(labels)),
    y=xes_peak.iloc[:, 1],
    yerr=xes_peak.iloc[:, 2],
    c=colors[0],
    fmt="o-",
    linewidth=1,
    capsize=4,
)

ax.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax.set_ylabel(r"Peak Positions (eV)", fontsize=11, labelpad=10.0)
ax.set_ylim(6492.0, 6493.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.1))
formatter = ticker.ScalarFormatter(useOffset=6490)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)

arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}
ax.annotate(
    r" ", xy=(0.2, 0.5), xycoords="axes fraction", xytext=(0, 0.4), textcoords="axes fraction", arrowprops=arrowprops
)

ax2 = ax.twinx()
ax2.set_position((0.2, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.errorbar(
    x=np.arange(len(labels)),
    y=xes_area.iloc[:, 7],
    yerr=xes_area.iloc[:, 8],
    c=colors[1],
    fmt="s-",
    linewidth=1,
    capsize=4,
    alpha=0.5,
)
ax2.set_xlim(-0.5, 11.5)

ax2.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax2.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax2.set_ylabel(r"Local Magnetic Moment ($\mathrm{\mu _B}$)", fontsize=11)  # Total Magnetization
ax2.set_ylim(2.5, 5.5)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0.1))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, left=False, right=True)

arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.5,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
}
ax2.annotate(
    r" ",
    xy=(0.8, 0.28),
    xycoords="axes fraction",
    xytext=(1.0, 0.35),
    textcoords="axes fraction",
    arrowprops=arrowprops,
)
ax.text(
    -0.18,
    1.0,
    r"b",
    transform=ax.transAxes,
    fontsize=14,
    va="center",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
)

plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_300_V6_3.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_3.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V6_3.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.gcf().set_facecolor("white")
plt.show()

### Version-2

#### 读取数据并 denoise, 得到峰和面积的 std, 后面再平均化， NMF 和 MCR

In [ ]:
# 读取数据文件夹
data_folder = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\ExSitu\αMnO2\Kbeta\Data\2023-CLAESS")  # noqa: RUF001
file_list = []
for item in data_folder.iterdir():
    if item.is_dir():
        file_dir = Path.joinpath(item, r"Mn")
        file_list.append(file_dir)

std_out = pd.DataFrame()
for file in file_list:
    path_out_A = Path.joinpath(path_out, f"{file.parts[-2]}", r'Mn')  # noqa: N816
    path_out_A.mkdir(parents=True, exist_ok=True)
    data = pd.DataFrame()
    for filetxt in file.glob("*.txt"):
        data_txt = pd.read_csv(filetxt, comment="#", sep=r"\s+", header=None)
        data = pd.concat(
            [data, data_txt],
            axis=1,
            ignore_index=True,
        )
    data = data.to_numpy()
    scp_data = NDDataset(
        data=data[:, 1::2].T,
        author="Cheng Liu",
        description="Kbeta of Mn, ALBA",
        history="creation",
    )
    scp_data.x = Coord(
        data[:, 0],
        name="Energy",
        units=ur.eV,
    )
    scp_data.y = Coord(
        np.arange((data[:, 1::2].shape[1])),
        name="numbers",
    )

    # PCA 重构数据
    recon_scp_data = scp.denoise(
        scp_data,
        ratio=99.8,
    )

    # 基线校准
    blc = scp.Baseline(
        log_level="INFO",
        model="polynomial",  # use a polynomial model
        order="linear",  # with linear method
        ranges=([6462.0, 6463.0], [6510.0, 6511.0]),
    )

    _ = blc.fit(scp_data)  # fit the baseline
    scp_baseline = blc.baseline
    scp_corrected = blc.corrected  # get the corrected dataset
    # scp_corrected.plot()
    # scp.plot_multiple(
    #     method="scatter",
    #     ms=5,
    #     datasets=[scp_baseline[1], scp_corrected[1], scp_data[1]],
    #     labels=["baseline", "corrected_data", 'average_raw'],
    #     legend="best",
    # )

    # 寻峰，以及 std 分布
    peakslist = [s.find_peaks(distance=10)[0].x.data for s in scp_corrected[:, 6490.0:6500.0]]
    pd.DataFrame(peakslist).to_csv(
        Path.joinpath(file, f"{file.parts[-2]}_all_peaks.csv"), index=False,header=[f"{file.parts[-2]}_peaks"],
    ) # type: ignore

    peakstd = pd.DataFrame(peakslist).std(ddof=0)
    # _ = peakslist.plot()

    # 计算面积，以及 std 分布
    inttrapz_area = scp_corrected.trapezoid(dim="x")
    # intsimps_area = scp_corrected.simpson(dim="x")
    pd.DataFrame(inttrapz_area.data).to_csv(
        Path.joinpath(file, f"{file.parts[-2]}_all_areas.csv"), index=False, header=[f"{file.parts[-2]}_areas"]
    ) # type: ignore
    inttrapz_area_std = pd.DataFrame(inttrapz_area.data).std(ddof=0) / np.mean(inttrapz_area.data)

    # scp.plot_multiple(
    #     method="scatter",
    #     ms=5,
    #     datasets=[inttrapz_area,  intsimps_area],
    #     labels=["trapzoidal rule", "simpson' rule"],
    #     legend="best",
    # )
    std = pd.concat(
        [peakstd, inttrapz_area_std],
        axis=1,
        ignore_index=True,
    )
    std_out = pd.concat(
        [std_out, std],
        axis=0,
        ignore_index=True,
    )

    # Evolving Factor Analysis (EFA) 计算
    efa = scp.EFA()
    efa.fit(recon_scp_data)
    efa.n_components = 2
    # C0 = efa.transform()
    # _ = C0.T.plot()
    St = efa.get_components()
    # _ = St.plot(title="components", legend=St.k.labels)

    mcr = scp.MCRALS(
        max_iter=100,
        normSpec="euclid",
        tol=0.0001,
        maxdiv=200,
        nonnegConc="all",
        nonnegSpec="all",
    )
    mcr.fit(recon_scp_data, St)
    # _ = mcr.C.T.plot()
    # _ = (mcr.St[1]/mcr.St[1].max()).plot()
    # _ = (St[1]/St[1].max()).plot(clear=False)
    # _ = (scp.mean(scp_data.T, dim='y')/scp.mean(scp_data.T, dim='y').max()).plot(clear=False)
    St[1].write_csv(
        Path.joinpath(path_out_A, f"{file.parts[-2]}_NMF.csv"),
    )
    scp.mean(scp_data.T, dim="y").write_csv(
        Path.joinpath(path_out_A, f"{file.parts[-2]}_MEAN.csv"),
    )
    mcr.St[1].write_csv(
        Path.joinpath(path_out_A, f"{file.parts[-2]}_MCR.csv"),
    )

std_out.to_csv(
    Path.joinpath(path_out, r"all_peak_std.csv"),
    index=False,
    header=[r"peak_std", r"area_std"],
)
print(r"Done")

#### 平均化， NMF 和 MCR 数据 去背景，归一化，计算 IDA

In [ ]:
# 读取 std 文件
std_file = list(path_out.glob(r"*_std.csv"))
df_std = pd.read_csv(std_file[0], index_col=None, header=0)

# 读取所有的 NMF 文件路径
filename = r"MEAN"
filelist2 = []
for item in path_out.iterdir():
    if item.is_dir():
        file_dir = list(Path.joinpath(item, r"Mn").glob(f"*_{filename}.csv"))
        filelist2.append(file_dir)

# 将所有 Raw Data 写入 NDDataset
data = pd.DataFrame()
for file in filelist2:
    df = pd.read_csv(file[0], index_col=None, header=0)
    data = pd.concat(
        [data, df],
        axis=1,
        ignore_index=True,
    )
data = data.to_numpy()
scp_data = NDDataset(
    data=data[:, 1::2].T,
    author="Cheng Liu",
    description="Kbeta of Mn, ALBA",
    history="creation",
    title="Count",
)
scp_data.x = Coord(
    data[:, 0],
    title="Energy",
    units=ur.eV,
)
scp_data.y = Coord(
    np.arange((data[:, 1::2].shape[1])),
    title="numbers",
)
# scp_data.plot()

# 基线校准
blc = scp.Baseline(
    log_level="INFO",
    model="polynomial",  # use a polynomial model
    order="linear",  # with linear method
    # ranges=([6462., 6465.], [6505., 6511.]),
    ranges=([6462.0, 6463.0], [6510.0, 6511.0]),
)

_ = blc.fit(scp_data)  # fit the baseline
scp_baseline = blc.baseline
scp_corrected = blc.corrected  # get the corrected dataset

# scp.plot_multiple(
#     method="scatter",
#     ms=5,
#     datasets=[scp_baseline[1], scp_corrected[1], scp_data[1]],
#     labels=["baseline", "corrected_data", 'average_raw'],
#     legend="best",
# )

# 计算面积
inttrapz_area = scp_corrected.trapezoid(dim="x")
# intsimps_area = scp_corrected.simpson(dim="x")

# scp.plot_multiple(
#     method="scatter",
#     ms=5,
#     datasets=[inttrapz_area,  intsimps_area],
#     labels=["trapzoidal rule", "simpson' rule"],
#     legend="best",
# )

# 归一化
for i in range(scp_corrected.shape[0]):
    scp_corrected[i, :] = np.divide(scp_corrected[i, :], inttrapz_area[i])
scp_corrected.to_xarray().to_pandas().T.to_csv(
    Path.joinpath(path_out, f"all_spect_{filename}.csv"),
    header=True,
)
# _ = scp_corrected.plot(
#     lw=1.0,
#     figure_figsize=(3.3, 2.5),
#     clear=True,
# )

# 寻峰
peakslist = [
    s.find_peaks(
        distance=10,
    )[0].x.data
    for s in scp_corrected[:, 6490.0:6500.0]
]
peakslist = pd.DataFrame(peakslist)
# _ = peakslist.plot(lw=1.0)

pd.concat(
    [peakslist, df_std["peak_std"]],
    axis=1,
    ignore_index=True,
).to_csv(
    Path.joinpath(path_out, f"all_peak_std_{filename}.csv"),
    index=False,
    header=[r"peak", r"std"],
)
# 面积
std = np.divide(df_std['area_std'].to_numpy(), inttrapz_area.data)
diff_std = np.sqrt(std[:]**2 + std[2]**2)

MNO = scp_corrected[2].copy()
diff = scp_corrected[:] - MNO
diff_area = diff.abs().trapezoid(dim="x")
pd.concat(
    [pd.DataFrame(diff_area.data), df_std["area_std"]],
    axis=1,
    ignore_index=True,
).to_csv(
    Path.joinpath(path_out, f"all_area_std_{filename}.csv"),
    index=False,
    header=[r"area", r"std"],
)
# _ = diff_area.plot()

In [ ]:
# 磁矩转换
from uncertainties import ufloat

# Load CSV
df = pd.read_csv(Path.joinpath(path_out, f"all_area_std_{filename}.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(  # noqa: E501
    how="all", axis=1
)


# Define the function, but only take one argument: the row
def calculate_ratios(row, data):
    Mn2 = ufloat(data.iloc[2, 0], data.iloc[2, 1])
    Mn4 = ufloat(data.iloc[4, 0], data.iloc[4, 1])

    A = ufloat(row["area"], row["std"])
    X = 3 + (A - Mn4) * 2 / (Mn2 - Mn4)

    return pd.Series({
        "X": X.nominal_value,
        "X_err": X.std_dev,
    })


# Apply with an extra argument using a lambda
results = df.apply(lambda row: calculate_ratios(row, df), axis=1)

# Combine and save
final_df = pd.concat([df, results], axis=1)
final_df.to_csv(Path.joinpath(path_out, f"all_area_std_{filename}.csv"), sep=",", header=True, index=False)

In [ ]:
spectrum = pd.read_csv(Path.joinpath(path_out, f"all_spect_{filename}.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(  # noqa: E501
    how="all", axis=1
)
xes_peak = pd.read_csv(Path.joinpath(path_out, f"all_peak_std_{filename}.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(  # noqa: E501
    how="all", axis=1
)
xes_area = pd.read_csv(Path.joinpath(path_out, f"all_area_std_{filename}.csv"), sep=",", index_col=None, skiprows=0, header=0).dropna(  # noqa: E501
    how="all", axis=1
)

diff_spectrum = spectrum.copy(deep=True)
odd_columns = diff_spectrum.columns[1:]
diff_spectrum[odd_columns] = diff_spectrum[odd_columns].sub(diff_spectrum.iloc[:, 5], axis=0)

In [ ]:
# 画图
%matplotlib inline

fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

labels = [
    r"R1_MnOOH",
    r"R2_ZnMn2O4",
    r"R3_MnO",
    r"R4_Mn2O3",
    r"R5_MnO2",
    r"S1_pristine",
    r"S2_1stDisch",
    r"S3_1stHCh_1p53V",
    r"S4_1stHCh_1p63V",
    r"S5_1stCh",
    r"S6_2ndDisch_1p3V",
    r"S7_2ndDisch",
]
colormap = ListedColormap(mpl.colormaps["sunset"](np.linspace(0, 1, spectrum.shape[1] - 1)), name=r"colormap")

# 图 A: Spectrum
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

# 多线叠加
for i in range(spectrum.shape[1] - 1):
    ax.plot(
        spectrum.iloc[:, 0],
        spectrum.iloc[:, 1 + i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],
        zorder=2,
        alpha=1 - 0.01 * i,
    )
    ax.plot(
        diff_spectrum.iloc[:, 0],
        diff_spectrum.iloc[:, 1 + i] - 0.01,
        lw=1,
        label=None,
        color=colormap.colors[i],
        zorder=2,
        alpha=1 - 0.01 * i,
    )

ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.set_xlim(6465, 6510)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5, offset=0))
ax.set_ylabel(ylabel=r"Relative Intensity (arb.u.)", fontsize=11, labelpad=0.0)
ax.set_ylim(-0.04, 0.16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.01, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.5,
)

axins = ax.inset_axes((0.78, 0.32, 0.2, 0.65))
for i in range(spectrum.shape[1] - 1):
    axins.plot(
        spectrum.iloc[:, 0],
        spectrum.iloc[:, 1 + i],
        lw=1,
        label=labels[i],
        color=colormap.colors[i],  # type: ignore
        zorder=0,
        alpha=1 - 0.01 * i,
    )
axins.set_xlim(6491, 6495)
axins.set_axis_off()
ax.text(
    -0.22,
    1.0,
    r"a",
    weight="bold",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=13,
)

# 图 B，Peak, area + std
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.2, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.errorbar(
    x=np.arange(len(labels)),
    y=xes_peak.iloc[:, 0],
    yerr=xes_peak.iloc[:, 1],
    c=colors[0],
    fmt="o-",
    linewidth=1,
    capsize=4,
)

ax.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax.set_ylabel(r"Peak Positions (eV)", fontsize=11, labelpad=10.0)
ax.set_ylim(6492.0, 6493.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.1))
formatter = ticker.ScalarFormatter(useOffset=6490)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)

arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}
ax.annotate(
    r" ", xy=(0.2, 0.5), xycoords="axes fraction", xytext=(0, 0.4), textcoords="axes fraction", arrowprops=arrowprops
)

ax2 = ax.twinx()
ax2.set_position((0.2, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.errorbar(
    x=np.arange(len(labels)),
    y=xes_area.iloc[:, 2],
    yerr=xes_area.iloc[:, 3],
    c=colors[1],
    fmt="s-",
    linewidth=1,
    capsize=4,
    alpha=0.5,
)
ax2.set_xlim(-0.5, 11.5)

ax2.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax2.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax2.set_ylabel(r"Local Magnetic Moment ($\mathrm{\mu _B}$)", fontsize=11)  # Total Magnetization
ax2.set_ylim(2.5, 5.5)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0.1))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, left=False, right=True)

arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.5,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
}
ax2.annotate(
    r" ",
    xy=(0.8, 0.28),
    xycoords="axes fraction",
    xytext=(1.0, 0.35),
    textcoords="axes fraction",
    arrowprops=arrowprops,
)
ax.text(
    -0.18,
    1.0,
    r"b",
    transform=ax.transAxes,
    fontsize=14,
    va="center",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
)

plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_300_V2.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"XES_Figure_600_V2.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 画图

In [ ]:
path_file = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\ExSitu\αMnO2\Kbeta\Results\2023-CLAESS\Results\V2")  # noqa: E501, RUF001

xes_peakarea = pd.read_csv(
    Path.joinpath(path_file, r"peak_area_std1.csv"), comment="#", sep=r",", header=0, index_col=None
)
xes_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"all_mean_normal.csv"), comment="#", sep=r",", header=0, index_col=None
)

In [ ]:
xes_peak = xes_peakarea.iloc[2:, 0:3]  # FD 数据
xes_area = xes_peakarea.iloc[2:, [0, 5, 6]]  # FD 数据
xes_spectrum = xes_spectrum.iloc[:, [0, *list(range(3, xes_spectrum.shape[1]))]]
diff_spectrum = xes_spectrum.copy(deep=True)
diff_spectrum.iloc[:, 1:] = diff_spectrum.iloc[:, 1:].sub(diff_spectrum.iloc[:, 3], axis=0)

In [ ]:
# 画图
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0.0, hspace=0.0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

labels = [
    r"$\mathrm{Ref.MnO}$",
    r"$\mathrm{Ref.Mn_2O_3}$",
    r"$\mathrm{Ref.MnO_2}$",
    r"$\mathrm{Pristine}$",
    r"$\mathrm{1^{st} \ Discharge}$",
    r"$\mathrm{1^{st} \ HalfCharge\#1}$",
    r"$\mathrm{1^{st} \ HalfCharge\#2}$",
    r"$\mathrm{1^{st} \ FullCharge}$",
    r"$\mathrm{2^{nd} \ HalfDischarge}$",
    r"$\mathrm{2^{nd} \ FullDischarge}$",
]

lss = [r"--", r"--", r"--", r"-", r"-",r"-",r"-",r"-",r"-",r"-"]
colormap = ListedColormap(mpl.colormaps["sunset"](np.linspace(0.0, 0.5, xes_spectrum.shape[1] - 1)), name=r"colormap")

# 多线叠加
for i in range(xes_spectrum.shape[1] - 1):
    ax.plot(
        xes_spectrum.iloc[:, 0],
        xes_spectrum.iloc[:, 1 + i],
        lw=1,
        ls=lss[i],
        label=labels[i],
        color=colors[i%8], # type: ignore
        zorder=5,
        alpha=1 - 0.01 * i,
    )
    ax.plot(
        xes_spectrum.iloc[:, 0],
        diff_spectrum.iloc[:, 1 + i] - 0.01,
        lw=1,
        ls=lss[i],
        color=colors[i%8], # type: ignore
        zorder=5,
        alpha=1 - 0.01 * i,
    )

ax.set_xlabel(r"Energy (eV)", fontsize=9)
ax.set_xlim(6465, 6510)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5, offset=0))

ax.set_ylabel(ylabel=r"Relative Intensity (arb.u.)", fontsize=9)
ax.set_ylim(-0.04, 0.16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.04))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.02))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(1.0, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
    columnspacing=0.5,
    handlelength=3.0,
    handletextpad=0.5,
)

axins = ax.inset_axes((0.68, 0.4, 0.30, 0.55))
for i in range(xes_spectrum.shape[1] - 1):
    axins.plot(
        xes_spectrum.iloc[:, 0],
        xes_spectrum.iloc[:, 1 + i],
        lw=1,
        label=labels[i],
        ls=lss[i],
        color=colors[i%8], # type: ignore
        zorder=0,
        alpha=1 - 0.01 * i,
    )
axins.set_xlim(6491, 6495)
axins.set_axis_off()
axins.set(xticks=[], xlabel=None, yticks=[], ylabel=None)
ax.text(
    0.32,
    0.40,
    r"$\mathrm{K}\beta^{\prime}$",
    transform=ax.transAxes,
    fontsize=10,
    va="center",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.65,
    0.93,
    r"$\mathrm{K}_{\beta 1,3}$",
    transform=ax.transAxes,
    fontsize=10,
    va="center",
    ha="right",
    fontfamily="Arial",
)

ax.text(
    -0.20,
    1.0,
    r"a",
    transform=ax.transAxes,
    fontsize=13,
    va="center",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.errorbar(
    x=np.arange(len(labels)),
    y=xes_peak.iloc[:, 1],
    yerr=xes_peak.iloc[:, 2],
    c=colors[0], # type: ignore
    fmt="o-",
    linewidth=1,
    capsize=4,
)

ax.set_xlim(-0.5, len(labels)-0.5)
ax.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax.set_ylabel(r"$\mathrm{K}_{\beta 1,3} \ Position \ (eV)$", fontsize=9, labelpad=10.0)  # Total Magnetization
ax.set_ylim(6492.0, 6493.5)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
formatter = ticker.ScalarFormatter(useOffset=6490)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"} # type: ignore
ax.annotate(
    r" ", xy=(0.2, 0.8), xycoords="axes fraction", xytext=(0, 0.9), textcoords="axes fraction", arrowprops=arrowprops
)

ax2 = ax.twinx()
ax2.set_position((0.6, 0, 1.0, 1.0))
ax2.set_box_aspect(0.8)

ax2.errorbar( # type: ignore
    x=np.arange(len(labels)),
    y=xes_area.iloc[:, 1],
    yerr=xes_area.iloc[:, 2],
    c=colors[1], # type: ignore
    fmt="s-",
    linewidth=1,
    capsize=4,
    alpha=0.5,
)

ax2.set_xticks(np.arange(len(labels)), labels=labels)
plt.setp(ax2.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor", fontfamily="Arial", fontsize=9)

ax2.set_ylabel(r"Local Magnetic Moment ($\mathrm{\mu _B}$)", fontsize=9)  # Total Magnetization
ax2.set_ylim(2.5, 5.5)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0.1))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))

arrowprops = {"arrowstyle": "<-", "color": colors[1], "alpha": 0.5, "connectionstyle": "angle,angleA=0,angleB=90,rad=10"} # type: ignore
ax2.annotate( # type: ignore
    r" ", xy=(0.8, 0.17), xycoords="axes fraction", xytext=(1.0, 0.07), textcoords="axes fraction", arrowprops=arrowprops
)

ax.text(
    -0.2, 1.0, r"b", transform=ax.transAxes, fontsize=14, va="center", ha="right", fontfamily="Arial", fontweight="bold"
)

plt.savefig(
    Path.joinpath(path_out, r"PaperDos_FigureSI_01_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"PaperDos_FigureSI_01_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"PaperDos_FigureSI_01_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.savefig(
    Path.joinpath(path_out, r"PaperDos_FigureSI_01_900.svg"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=900,
)
plt.gcf().set_facecolor("white")
plt.show()